In [5]:
import os
import re
from pathlib import Path

def estandarizar_nombres_experimento(ruta_directorio, numero_exp, data_id=None, eliminar_fecha=True, simulacro=True):
    ruta = Path(ruta_directorio)
    
    if not ruta.exists():
        print(f"❌ Error: La ruta '{ruta}' no existe.")
        return

    print(f"--- Procesando: {ruta.name} (Simulacro: {simulacro}) ---")

    # 1. Patrón para FECHA (La usamos como ancla para saber dónde parar de borrar)
    regex_fecha_str = r'_\d{8}_\d{6}'
    patron_fecha = re.compile(regex_fecha_str)

    # 2. Patrón EXP + BASURA + CHUNK
    # Encuentra: exp_2_loquesea_chunk
    patron_con_chunk = re.compile(r'exp_\d+.*?_chunk')

    # 3. Patrón EXP + BASURA + FECHA
    # Encuentra: exp_2_loquesea_2025...
    # Este es el que te fallaba antes. Ahora busca hasta encontrar la fecha.
    patron_con_fecha = re.compile(r'exp_\d+.*?' + regex_fecha_str)

    # 4. Patrón simple (Fallback)
    patron_exp_simple = re.compile(r'exp_\d+')

    archivos_modificados = 0

    for archivo in ruta.iterdir():
        if archivo.is_file():
            nombre_viejo = archivo.name
            nombre_nuevo = nombre_viejo

            # --- A. CASO: TENEMOS UN DATA_ID PARA INSERTAR ---
            if data_id:
                
                # PRIORIDAD 1: Es un archivo tipo CHUNK
                if 'chunk' in nombre_nuevo and patron_con_chunk.search(nombre_nuevo):
                    # Reemplaza todo el bloque "exp_..._chunk" por el nuevo limpio
                    nombre_nuevo = patron_con_chunk.sub(
                        f'exp_{numero_exp}_{data_id}_chunk', 
                        nombre_nuevo
                    )

                # PRIORIDAD 2: No es chunk, pero TIENE FECHA
                # Aquí está el arreglo: Usamos la fecha como límite para borrar lo viejo (simu_1)
                elif patron_con_fecha.search(nombre_nuevo):
                    if eliminar_fecha:
                        # Si vamos a borrar la fecha, reemplazamos "exp_..._fecha" por "exp_..._id"
                        # Esto se "come" el _simu_1 y la fecha de golpe.
                        nombre_nuevo = patron_con_fecha.sub(
                            f'exp_{numero_exp}_{data_id}', 
                            nombre_nuevo
                        )
                    else:
                        # Si quieres mantener la fecha (raro, pero posible), habría que ajustar la regex
                        pass

                # PRIORIDAD 3: Ni chunk ni fecha reconocible (Fallback)
                else:
                    nombre_nuevo = patron_exp_simple.sub(
                        f'exp_{numero_exp}_{data_id}', 
                        nombre_nuevo
                    )

            # --- B. CASO: SOLO ACTUALIZAR NUMERO EXP (Sin Data_ID) ---
            else:
                 nombre_nuevo = patron_exp_simple.sub(f'exp_{numero_exp}', nombre_nuevo)

            # --- C. LIMPIEZA FINAL DE FECHA ---
            # (Solo si no fue eliminada ya en el paso A -> Prioridad 2)
            if eliminar_fecha and patron_fecha.search(nombre_nuevo):
                 nombre_nuevo = patron_fecha.sub('', nombre_nuevo)

            # --- EJECUTAR ---
            if nombre_nuevo != nombre_viejo:
                if simulacro:
                    print(f"📝 [SIM] {nombre_viejo} \n       -> {nombre_nuevo}")
                else:
                    try:
                        archivo.rename(ruta / nombre_nuevo)
                        print(f"✅ [OK] {nombre_nuevo}")
                    except OSError as e:
                        print(f"❌ [Error] {e}")
                
                archivos_modificados += 1

    print(f"\nArchivos afectados: {archivos_modificados}")

In [11]:
# 1. Definir la ruta
script_path = os.getcwd()
project_path = os.path.join(script_path, '..')

DATA_ID = 'kc_houses'
EXPERIMENT_ID = 'experiment_2'
folder_path = os.path.join(project_path, 'results', EXPERIMENT_ID, DATA_ID)

# 2. Ejecutar primero en modo SIMULACRO (recomendado)
# Esto cambiará 'exp_X' a 'exp_2' y borrará la fecha.
estandarizar_nombres_experimento(
    ruta_directorio=folder_path, 
    numero_exp=EXPERIMENT_ID.split('_')[-1], 
    data_id=DATA_ID,
    eliminar_fecha=True, 
    simulacro=False  # <--- Cambia a False cuando estés listo
)

--- Procesando: kc_houses (Simulacro: False) ---
✅ [OK] plot_results_exp_2_kc_houses.png
✅ [OK] results_exp_2_kc_houses.pkl
✅ [OK] results_exp_2_kc_houses_chunk_0.pkl
✅ [OK] results_exp_2_kc_houses_chunk_10.pkl
✅ [OK] results_exp_2_kc_houses_chunk_11.pkl
✅ [OK] results_exp_2_kc_houses_chunk_12.pkl
✅ [OK] results_exp_2_kc_houses_chunk_13.pkl
✅ [OK] results_exp_2_kc_houses_chunk_14.pkl
✅ [OK] results_exp_2_kc_houses_chunk_15.pkl
✅ [OK] results_exp_2_kc_houses_chunk_16.pkl
✅ [OK] results_exp_2_kc_houses_chunk_17.pkl
✅ [OK] results_exp_2_kc_houses_chunk_18.pkl
✅ [OK] results_exp_2_kc_houses_chunk_19.pkl
✅ [OK] results_exp_2_kc_houses_chunk_1.pkl
✅ [OK] results_exp_2_kc_houses_chunk_2.pkl
✅ [OK] results_exp_2_kc_houses_chunk_3.pkl
✅ [OK] results_exp_2_kc_houses_chunk_4.pkl
✅ [OK] results_exp_2_kc_houses_chunk_5.pkl
✅ [OK] results_exp_2_kc_houses_chunk_6.pkl
✅ [OK] results_exp_2_kc_houses_chunk_7.pkl
✅ [OK] results_exp_2_kc_houses_chunk_8.pkl
✅ [OK] results_exp_2_kc_houses_chunk_9.pkl

Archi